In [1]:
# 📦 Importations
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import WeightedRandomSampler

In [2]:
# 📂 Préparer les données
data_dir = "../data/raw/chest_Xray/test"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

# Dataset
train_dataset = datasets.ImageFolder(root="../data/raw/chest_Xray/train", transform=transform)
test_dataset = datasets.ImageFolder(root="../data/raw/chest_Xray/test", transform=transform)
val_dataset = datasets.ImageFolder(root="../data/raw/chest_Xray/val", transform=transform)

# train
class_counts = [train_dataset.targets.count(0), train_dataset.targets.count(1)]
class_weights = 1. / np.array(class_counts)
sample_weights = [class_weights[label] for label in train_dataset.targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)

# test
class_counts = [test_dataset.targets.count(0), test_dataset.targets.count(1)]
class_weights = 1. / np.array(class_counts)
sample_weights = [class_weights[label] for label in test_dataset.targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

test_loader = DataLoader(test_dataset, batch_size=32, sampler=sampler)

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

classes = train_dataset.classes

In [4]:
# 🚀 Charger MobileNet pré-entraîné et l'adapter
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 2)  # 2 classes : NORMAL et PNEUMONIA

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [10]:
# Charger le modèle et l'extracteur
extractor = AutoFeatureExtractor.from_pretrained("apple/mobilevit-small")
model = AutoModelForImageClassification.from_pretrained("apple/mobilevit-small")

# Adapter le classifieur à 2 classes
model.classifier = nn.Linear(model.classifier.in_features, 2)


NameError: name 'AutoModelForImageClassification' is not defined

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# ⚙️ Définir l'optimiseur et la fonction de perte
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Pour suivre Loss et Accuracy
train_losses = []
train_accuracies = []

In [ ]:
# 🏋️ Boucle d'entraînement
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.2f}%")

In [ ]:
torch.save(model.state_dict(), "mobilevit_pneumonia.pth")
